In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import os

In [4]:
GLOBAL_FLOAT_PRECISION_NUMPY = np.float64

In [5]:
torch.set_default_dtype(torch.float64)

## Data preparation

### Load data

In [6]:
chirp_direction = 'chirp_up'
data_file_path = r'C:\Users\vaish\dsvv_atomionics\compiled_data\compiled_' + f'{chirp_direction}.pkl'
data_cols = [
		'master_run_number', 'chirp_rate', 'fraction',
		'CA+_mean', 'CA+_std_dev', 'CA+_percentile_0', 'CA+_percentile_10', 'CA+_percentile_20', 
		'CA+_percentile_30', 'CA+_percentile_40', 'CA+_percentile_50', 'CA+_percentile_60', 'CA+_percentile_70', 
		'CA+_percentile_80', 'CA+_percentile_90', 'CA+_percentile_100', 'CA-_mean', 'CA-_std_dev', 'CA-_percentile_0', 
		'CA-_percentile_10', 'CA-_percentile_20', 'CA-_percentile_30', 'CA-_percentile_40', 'CA-_percentile_50', 
		'CA-_percentile_60', 'CA-_percentile_70', 'CA-_percentile_80', 'CA-_percentile_90', 'CA-_percentile_100'
	]

data = pd.DataFrame(pd.read_pickle(data_file_path), columns=data_cols, dtype=GLOBAL_FLOAT_PRECISION_NUMPY)
data = data.astype({'master_run_number': 'int32'})

# 80-20 train/test split
train_data_df = data.sample(frac=0.8, random_state=0)
test_data_df = data.drop(train_data_df.index)

print(train_data_df.shape)
# print(train_data_df.head())
print(test_data_df.shape)
# print(test_data_df.head())

(126, 29)
(32, 29)


In [7]:
compiled_constants_csv_path = r'C:\Users\vaish\dsvv_atomionics\compiled_data\compiled_constants.csv'
compiled_constants_df = pd.read_csv(compiled_constants_csv_path)
compiled_constants_dtypes = {
	'master_run': 'int32',
	'chirp_direction': 'str',
	'bigT': GLOBAL_FLOAT_PRECISION_NUMPY,
	'Keff': GLOBAL_FLOAT_PRECISION_NUMPY,
	'contrast': GLOBAL_FLOAT_PRECISION_NUMPY,
	'g0': GLOBAL_FLOAT_PRECISION_NUMPY,
	'fringe_offset': GLOBAL_FLOAT_PRECISION_NUMPY
}
compiled_constants_df = compiled_constants_df.astype(compiled_constants_dtypes)

print(compiled_constants_df.shape)
# print(compiled_constants_df.head())

(4, 7)


## Converting chirp rates to fractions

In [8]:
def sine(alpha, Keff, bigT, contrast, g0, fringe_offset):
    phi = (Keff * g0 - 2 * torch.pi * alpha) * (bigT**2)
    return (-contrast * torch.cos(phi) + fringe_offset)

In [9]:
def get_predicted_fractions_from_outputs(outputs, data_df):
	predicted_fractions = []
	for index in range(outputs.shape[0]):
		data_row = data_df.iloc[index]
		master_run = data_row['master_run_number']
		compiled_constants_row = compiled_constants_df[(compiled_constants_df['master_run'] == master_run) & (compiled_constants_df['chirp_direction'] == chirp_direction)]

		Keff = compiled_constants_row['Keff'].item()
		bigT = compiled_constants_row['bigT'].item()
		contrast = compiled_constants_row['contrast'].item()
		g0 = compiled_constants_row['g0'].item()
		fringe_offset = compiled_constants_row['fringe_offset'].item()
		current_output = outputs[index]

		predicted_fraction = sine(current_output, Keff, bigT, contrast, g0, fringe_offset)
		predicted_fractions.append(predicted_fraction)
	
	predicted_fractions_tensor = torch.stack(predicted_fractions, 0)
	return predicted_fractions_tensor


In [10]:
# A little test you can run, to verify that gradients do propagate through this transformation
# https://stackoverflow.com/questions/70426391/how-to-transform-output-of-nn-while-still-being-able-to-train

# start with some random tensor representing the input predictions
# make sure it requires_grad
pred = torch.rand((4, 5, 2, 3)).requires_grad_(True)
# transform it
tpred = get_predicted_fractions_from_outputs(pred, train_data_df)

# make up some "default" loss function and back-prop
tpred.mean().backward()

# check to see all gradients of the original prediction:
pred.grad

tensor([[[[-3.0780e-07, -3.0785e-07, -3.0781e-07],
          [-3.0772e-07, -3.0776e-07, -3.0786e-07]],

         [[-3.0782e-07, -3.0778e-07, -3.0775e-07],
          [-3.0786e-07, -3.0773e-07, -3.0781e-07]],

         [[-3.0782e-07, -3.0786e-07, -3.0772e-07],
          [-3.0774e-07, -3.0775e-07, -3.0771e-07]],

         [[-3.0770e-07, -3.0775e-07, -3.0778e-07],
          [-3.0779e-07, -3.0785e-07, -3.0779e-07]],

         [[-3.0776e-07, -3.0778e-07, -3.0786e-07],
          [-3.0774e-07, -3.0781e-07, -3.0772e-07]]],


        [[[-3.0777e-07, -3.0776e-07, -3.0779e-07],
          [-3.0771e-07, -3.0786e-07, -3.0778e-07]],

         [[-3.0774e-07, -3.0775e-07, -3.0779e-07],
          [-3.0774e-07, -3.0786e-07, -3.0783e-07]],

         [[-3.0771e-07, -3.0784e-07, -3.0784e-07],
          [-3.0781e-07, -3.0780e-07, -3.0779e-07]],

         [[-3.0774e-07, -3.0777e-07, -3.0775e-07],
          [-3.0785e-07, -3.0775e-07, -3.0778e-07]],

         [[-3.0777e-07, -3.0779e-07, -3.0781e-07],
          [

## MSLE Loss

In [11]:
def MSLELoss(pred, target):
    return torch.mean((torch.log1p(pred) - torch.log1p(target)) ** 2)

## Benchmark MSE score

In [12]:
test_chirp_rates = torch.tensor(test_data_df['chirp_rate'].values)
test_fractions = torch.tensor(test_data_df['fraction'].values)

In [13]:
def get_baseline_score(predicted_tensor, target_tensor, loss_fn):
	loss = loss_fn(predicted_tensor, target_tensor)
	return loss.item()

In [14]:
predicted_fractions_tensor = get_predicted_fractions_from_outputs(test_chirp_rates, test_data_df)

MSELoss = nn.MSELoss()
mse_baseline_score = get_baseline_score(predicted_fractions_tensor, test_fractions, MSELoss)
print(f'MSE baseline score: {mse_baseline_score}')

msle_baseline_score = get_baseline_score(predicted_fractions_tensor, test_fractions, MSLELoss)
print(f'MSLE baseline score: {msle_baseline_score}')

MSE baseline score: 0.006329568853331391
MSLE baseline score: 0.002648836092938011


## Training

### Neural Network

In [15]:
class TransformationNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(TransformationNN, self).__init__()
        # Define the layers of the network: 2 fully connected layers 27 -> 14 -> 7 -> 1
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, output_dim)
    
    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = self.fc3(x)
        return x

In [16]:
training_data_tensor = torch.tensor(train_data_df.drop(columns=['fraction', 'master_run_number']).values)
observed_fractions_tensor = torch.tensor(train_data_df['fraction'].values).view(-1, 1)

In [22]:
model = TransformationNN(input_dim=27, output_dim=1)
optimizer = optim.Adam(model.parameters(), lr=0.01)
MSELoss = nn.MSELoss()

In [18]:
def train(model, optimizer, loss_fn, epochs, training_data_tensor, observed_fractions_tensor):
	loss_values = []
	for epoch in range(epochs):
		model.train()

		# Forward pass: Compute predicted y by passing x to the model
		outputs = model(training_data_tensor)
		predicted_tensors = get_predicted_fractions_from_outputs(outputs, train_data_df)

		loss = loss_fn(predicted_tensors, observed_fractions_tensor)
		loss_values.append(loss.item())

		optimizer.zero_grad()
		loss.backward()
		optimizer.step()

		if epoch % (epochs / 10) == 0:
			print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item()}')
	
	plt.figure(figsize=(10, 6))
	plt.plot(range(epochs), loss_values, label='Training Loss')
	plt.xlabel('Epochs')
	plt.ylabel('Loss')
	plt.title('Training Loss vs. Epochs')
	plt.legend()
	plt.show()


In [23]:
train(model, optimizer, MSELoss, 1000, training_data_tensor, observed_fractions_tensor)

Epoch [1/1000], Loss: 0.01597793400946705
Epoch [101/1000], Loss: 0.015448606962278644
Epoch [201/1000], Loss: 0.014943221666267778


KeyboardInterrupt: 

## Evaluation

In [20]:
def get_model_score(predicted_tensor, target_tensor, loss_fn):
	loss = loss_fn(predicted_tensor, target_tensor)
	return loss.item()

In [21]:
test_chirp_rates_with_noise_tensor = torch.tensor(test_data_df.drop(columns=['fraction', 'master_run_number']).values, dtype=torch.float64)

# Set model to evaluation mode
model.eval()

# Forward pass through the model
with torch.no_grad():
    corrected_chirp_rates = model(test_chirp_rates_with_noise_tensor)
    predicted_fractions = get_predicted_fractions_from_outputs(corrected_chirp_rates, test_data_df)

test_fractions_shape_corrected = test_fractions.view(-1, 1)
mse_model_score = get_model_score(predicted_fractions, test_fractions_shape_corrected, MSELoss)
print(f'MSE model score: {mse_model_score}')

msle_model_score = get_model_score(predicted_fractions, test_fractions_shape_corrected, MSLELoss)
print(f'MSLE model score: {msle_model_score}')

MSE model score: 0.0072881096073342515
MSLE model score: 0.0030700739889508383
